In [ ]:
from io import BytesIO
import pandas as pd
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

In [ ]:
from google.cloud import storage
storage_client = storage.Client()

In [ ]:
bucket_name = 'army-tribunals'
bucket = storage_client.get_bucket(bucket_name)

In [ ]:
prefix = 'embeddings/'
blobs = storage_client.list_blobs(bucket_name,prefix=prefix)
csv_path = 'embeddings/clustered_documents.csv'
blob = bucket.blob(csv_path)
file_obj = blob.download_as_bytes()

str_obj = BytesIO(file_obj)
df = pd.read_csv(str_obj)

In [ ]:
df.head()

,Unnamed: 0,case_number,court,year,month,pdf_url,civilian_judge,military_judge,pdf_text,embedding,cluster
0,1,OA_2509_of_2012,CHANDIGARH,2013,11,https://www.aftdelhi.nic.in/benches/chandigarh...,prakash krishna,lt. gen. hs panag,"ARMED FORCES TRIBUNAL, CHANDIGARH \nREGIONAL B...",[-1.58602800e-02 1.72294155e-02 -1.17799863e-...,0
1,2,TA_47_of_2013,CHANDIGARH,2013,11,https://www.aftdelhi.nic.in/benches/chandigarh...,prakash krishna,lt. gen. hs panag,"ARMED FORCES TRIBUNAL, CHANDIGARH \nREGIONAL B...",[-0.00136113 -0.00543779 -0.00315412 ... -0.02...,0
2,3,OA_1358_of_2012,CHANDIGARH,2013,12,https://www.aftdelhi.nic.in/benches/chandigarh...,prakash krishna,lt. gen. hs panag,"ARMED FORCES TRIBUNAL, CHANDIGARH \nREGIONAL B...",[-0.01257746 0.00685315 -0.00614937 ... -0.02...,0
3,4,TA_833_of_2010,CHANDIGARH,2013,12,https://www.aftdelhi.nic.in/benches/chandigarh...,prakash krishna,lt. gen. hs panag,"ARMED FORCES TRIBUNAL, CHANDIGARH \nREGIONAL B...",[-0.01298816 -0.00649069 0.00348066 ... -0.01...,0
4,5,OA_657_of_2010,CHANDIGARH,2014,1,https://www.aftdelhi.nic.in/benches/chandigarh...,prakash krishna,lt. gen. hs panag,"ARMED FORCES TRIBUNAL, CHANDIGARH \nREGIONAL B...",[ 0.00575938 -0.00025969 -0.00300858 ... -0.01...,0


In [ ]:
df = df.dropna(subset=['embedding'])
df["embedding"] = df["embedding"].apply(lambda x: np.fromstring(x[1:-1], sep=" "))

In [ ]:
n_clusters = 10

In [ ]:
import spacy
import re
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

nlp = spacy.load("en_core_web_sm")
stop_words = STOP_WORDS
parser = English()

def preprocess_text(text):

    text = re.sub(r'\S+@\S+', '', text)
    text = re.sub(r'http\S+', '', text)
    
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    tokens = parser(text)
    tokens = [token.lemma_.lower().strip() if token.lemma_ != "-PRON-" else token.lower_ for token in tokens]
    tokens = [token for token in tokens if token not in stop_words]

    text = " ".join(tokens)
    
    return text


In [ ]:
#Topic Modelling on Cluster 2
df_cluster_2=df[df['cluster']==2]

In [ ]:
from nltk.tokenize import word_tokenize
from gensim import corpora
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import gensim


def preprocess_text(text):
 
    tokens = word_tokenize(text)
    tokens = [token.lower() for token in tokens]
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    tokens = [token for token in tokens if token.isalpha()]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return tokens

cluster_docs = [preprocess_text(doc) for doc in df_cluster_2['pdf_text']]

dictionary = corpora.Dictionary(cluster_docs)
corpus = [dictionary.doc2bow(doc) for doc in cluster_docs]

In [ ]:
num_topics = 10
chunksize = 2000
passes = 20
iterations = 400
eval_every = None
n_clusters = 5

    
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                                id2word=dictionary,
                                                num_topics=num_topics,
                                                chunksize=chunksize,
                                                passes=passes,
                                                iterations=iterations,
                                                eval_every=eval_every)


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models

# Visualize the LDA model
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
pyLDAvis.display(vis)


/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.9/dist-packages/pyLDAvis/_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


In [ ]:
df_cluster_1=df[df['cluster']==1]

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
#Topic Modelling for Cluster 5
cluster_docs = [preprocess_text(doc) for doc in df_cluster_5['pdf_text']]

dictionary = corpora.Dictionary(cluster_docs)
corpus = [dictionary.doc2bow(doc) for doc in cluster_docs]

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Visualize the LDA model
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
pyLDAvis.display(vis)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.9/dist-packages/pyLDAvis/_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


In [ ]:
df['pdf_text'][0]

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'ARMED FORCES TRIBUNAL, CHANDIGARH \nREGIONAL BENCH AT CHANDIMANDIR \n-.- \n\nOA 2509 of 2012 \n\nSmt. Harjinder Kaur \n\n……                Petitioner(s) \n\nVs \n\nUnion of India and others \n\n……                Respondent(s)  \n\n-.- \n\nFor the Petitioner (s)      :  \n\nMr. Rajeev Anand, Advocate  \n\nFor the Respondent(s)   : \n\nMrs. Sangeeta Dubey, CGC. \n\nCoram:  Justice Prakash Krishna, Judicial Member. \n\nLt Gen (Retd) HS Panag, Administrative Member. \n\n-.- \n\nORDER \n   15-11-2013 \n-.- \n\nThis application has been filed by the petitioner who is the widow of late \n\nLance  Naik  Malkit  Singh,  under  Section  14  of  the  Armed  Forces  Act.  2007 \n\nseeking the following reliefs:- \n\n(i)   For  quashing  of  the  order  dated19-05-2012  (Annexure  A-9) \n\nwherein  rejection  has  been  made  to  the  claim  of  the  applicant \n\nfor declaration of the death of her husband as a Battle Casualty \n\nand  grant  of  all  consequential  benefits  like  the  Liberalis

In [ ]:
import openai
import tiktoken
from langchain.chains import AnalyzeDocumentChain
from langchain.chains.summarize import load_summarize_chain
from langchain.llms import OpenAI

api_key = os.environ['OPENAI_API_KEY']

model = OpenAI(api_key=api_key, temperature=0)
summary_chain = load_summarize_chain(llm=model, chain_type='map_reduce')

summarize_document_chain = AnalyzeDocumentChain(combine_docs_chain=summary_chain)
result = summarize_document_chain.run(pages)
print('output(AnalyzeDocumentChain):', result)

In [ ]:
result

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


' This case discusses the discharge of Kuldip Singh from service due to permanent low medical category. The petitioner argued that a similar case of discharge was set aside by the Tribunal and that he was provided a sheltered appointment to complete qualifying service for invalid pension. The discharge was ordered under Army Rule 13 (3) Item III (v). The Tribunal set aside the discharge and ordered the petitioner to be reinstated in service from the date of discharge, with no pay and allowances for the period between discharge and rejoining service.'

In [ ]:

# Run the analyze document chain on your pages
result = summarize_document_chain.run(pages)
print('output(AnalyzeDocumentChain):', result)


/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


output(AnalyzeDocumentChain):  Gurtej Singh, a petitioner, is seeking to quash an order from the Union of India and others that rejected his claim for disability pension. He had suffered a gun-shot injury while on duty in 2005 and was discharged from service in 2008. The Court of Inquiry found that the injury was attributable to military service and the petitioner was held entitled to disability pension. The Honble High Court quashed the impugned order and directed the respondents to assess and release the disability pension in favour of the petitioner for 20% disability from the date of his discharge.


In [ ]:
result = summarize_document_chain.run(pages)
print('output(AnalyzeDocumentChain):', result)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


output(AnalyzeDocumentChain):  This case involves Gulzari Lal, a petitioner who was invalided out of the Army due to Idiopathic Epilepsy in 1969 and was granted disability pension at the rate of 30% permanent. In 2006, the petitioner was called for a Review Medical Board which assessed his disability at less than 20%, resulting in the discontinuation of his disability element. The Hon'ble High Court of Himachal Pradesh directed the respondents to grant disability pension as per rules and the petitioner was entitled to continue receiving the disability element of disability pension. The respondents were to issue necessary amendments and recommence payment of the disability element within three months of receipt of the order.


In [ ]:
#Looking at cluster 1
pages= df_cluster_1.iloc[678]['pdf_text']

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
#115
result = summarize_document_chain.run(pages)
print('output(AnalyzeDocumentChain):', result)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


output(AnalyzeDocumentChain):  Tarsem Singh filed a petition under Section 14 of the Armed Forces Tribunal Act, 2007, seeking a direction to the respondents to release the revised pension in the rank of Honorary Naib Subedar w.e.f. 01.01.2006. The Supreme Court dismissed the respondents' appeal and allowed the OA, directing the respondents to release the service pension to the applicant in the rank of Naib Subedar w.e.f. 01.01.2006 within a period of three months. The Supreme Court also clarified that no interest shall be payable on arrears in such cases.


In [ ]:
#67
result = summarize_document_chain.run(pages)
print('output(AnalyzeDocumentChain):', result)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


output(AnalyzeDocumentChain):  This case involves a pre-01.01.2006 retiree who was granted the Honorary rank of Naib Subedar. The Supreme Court held the petitioner entitled to the benefit from 01.01.2006, without interest. The OA was allowed and disposed of in terms of the observations and directions given, subject to verification of the factual matrix by the respondents. The respondents were directed to release the service pension to the applicant in the rank of Naib Subedar w.e.f. 01.01.2006 within a period of four months from the date of receipt of a certified copy of this ORDER. No interest on the arrears shall be admissible, however, in case compliance is not made by the respondents within the stipulated time, interest @ 8% shall accrue to the petitioner from the date of this order.


In [ ]:
#678
result = summarize_document_chain.run(pages)
print('output(AnalyzeDocumentChain):', result)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


output(AnalyzeDocumentChain):  The applicant is seeking a direction to the respondents to release the revised pension in the rank of Honorary Naib Subedar w.e.f. 01.01.2006. The Supreme Court clarified that no interest shall be payable on arrears. The OA is allowed and disposed of with directions to the respondents to release the service pension to the applicant in the rank of Naib Subedar w.e.f. 01.01.2006 within a period of three months from the date of receipt of a certified copy of this order by the representative of the respondents. If the compliance is not made by the respondents within the stipulated time, interest @ 8% shall accrue to the applicant from the date of this order.


In [ ]:
result

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


' The applicant is seeking a direction to the respondents to release the revised pension in the rank of Honorary Naib Subedar w.e.f. 01.01.2006. The Supreme Court clarified that no interest shall be payable on arrears. The OA is allowed and disposed of with directions to the respondents to release the service pension to the applicant in the rank of Naib Subedar w.e.f. 01.01.2006 within a period of three months from the date of receipt of a certified copy of this order by the representative of the respondents. If the compliance is not made by the respondents within the stipulated time, interest @ 8% shall accrue to the applicant from the date of this order.'

In [ ]:
#Looking at cluster 4
df_cluster_4=df[df['cluster']==4]
pages= df_cluster_4.iloc[112]['pdf_text']

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
result = summarize_document_chain.run(pages)
print('output(AnalyzeDocumentChain):', result)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


output(AnalyzeDocumentChain):  In this petition, Suresh Chander Malhan, a former Army personnel, requested the benefit of broad-banding by rounding off the percentage of his disability. The respondents rejected the representation, but the petitioner argued that his case was covered by a Supreme Court judgment. The petition was allowed and the respondents were directed to calculate the arrears and release them to the petitioner within three months.
